In [1]:
import numpy as np
import random
#np.random.seed(123)

In [2]:
for i in range(3, -1, -1):
    print(i)

3
2
1
0


In [3]:
def areBinaryWeights(weights):
    flag = True
    th = 1e-12
    for i in range(np.shape(weights),-1,-1):
        if(weights[i] > th and (1-weights[i])>th):
            flag = false
            break
            
    return flag

In [4]:
import uuid
import datetime

def empty_problem():
    problem = {}
    problem['type'] = 'None'
    problem['N'] = 0
    problem['outliers'] = []
    problem['priors'] = []
    problem['dof'] = 0
    #problem['time'] = now
    problem['time'] = datetime.datetime.now()
    # Unique identifier
    problem['uuid'] = uuid.uuid4().hex
    return problem
    

In [5]:
def isProblem(problem):
    mandatory_fields = {'type', 'N', 'outliers', 'priors', 'dof'}
    #flag = all(isfield(problem, mandatory_fields))
    flag = all(field in problem for field in mandatory_fields)
    #flag = all(field in problem for field in mandatory_fields)
    return flag

In [6]:
def rand_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random. rand(n,m)
    
    return r

In [7]:
def randn_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random.randn(n,m)
    return r

In [8]:
from numpy.linalg import matrix_rank
import random

def random_matrix(n, m, desired_rank):
    max_rank = min(n, m)
    if desired_rank==None:
        desired_rank = max_rank
    assert desired_rank<=max_rank, 'Cannot generate a matrix a random matrix with the desired rank'
    M = 3*np.random.rand(n, m)
    while matrix_rank(M) < max_rank and matrix_rank(M) > 0:
        M = rand_in(n, m, [-1, 1])
    rank_M = matrix_rank(M)
    
    if rank_M != desired_rank:
        col_perm = random.sample(range(m), rank_M - desired_rank + 1)
        for i in range(rank_M-desired_rank+1):
                M[:, col_perm[0]] = M[:, col_perm[i]]
    return M

In [9]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    th1 = (mu+1)/mu*barc2
    th2 = (mu)/(mu+1)*barc2
    print('mu is: ', mu)
    print('th1 is: ', th1)
    print('th2 is: ', th2)
    
    for k in range(np.shape(residuals)):
        if residuals[k] - th1 >=0:
            weights[k] = 0
        elif residuals[k] - th2 <=0:
            weights[k] = 1
        else:
            weights[k] = np.sqrt(barc2*mu*(mu+1)/residuals[k]) - mu
            #assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!', weights(k)
            assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!'
  
    return weights, th1, th2     
    

In [10]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
import numpy as np
import numbers
import random
from scipy.stats import chi2

In [11]:
def linearRegressionProblem(N, outliers_percentage, **kwargs):
    
    StateDimension = kwargs.get('StateDimension', 3)
    print(StateDimension, type(StateDimension))
    print(isinstance(StateDimension, numbers.Number))
    print(np.size(StateDimension), np.size(StateDimension)==1) 
    print(StateDimension >=1) 
    
    assert isinstance(StateDimension, numbers.Number) and np.size(StateDimension)==1 and StateDimension >=1, 'StateDimension must be numeric, scalar, and >=1'
    
    MeasurementDimension = kwargs.get('MeasurementDimension', 3)
    assert isinstance(MeasurementDimension, numbers.Number) and np.size(MeasurementDimension)==1 and np.array(MeasurementDimension)>=1, 'MeasurementDimension must be numeric, scalar, and >=1'
    
    StateMeanandStd = kwargs.get('StateMeanAndStd', [1,1])
    assert all(isinstance(elem, numbers.Number) for elem in StateMeanandStd) and np.size(StateMeanandStd)==2 and all(np.array(StateMeanandStd) >=0), 'StateMeanandStd should be numeric, vector of length 2, and >=0'
    
    MeasurementNoiseStd = kwargs.get('MeasurementNoiseStd', 0.1)
    assert isinstance(MeasurementNoiseStd, numbers.Number) and np.size(MeasurementNoiseStd)==1 and MeasurementNoiseStd>0
    
    OutlierSpace = kwargs.get('OutlierSpace', [-100, 100])
    assert all(isinstance(elem, numbers.Number) for elem in OutlierSpace) and np.size(OutlierSpace)==2 and np.array(OutlierSpace[0]) <= np.array(OutlierSpace[1]), 'OutlierSpace must be numeric, vector of length 2, and OutlierSpace[0] < OutlierSpace[1]'
    
    if kwargs==None:
        outliers_percentage = 0
        
    problem = empty_problem()
    problem['N'] = N
    problem['StateDimension'] = StateDimension
    problem['MeasurementDimension'] = MeasurementDimension
    problem['MeasurementNoiseStd'] = MeasurementNoiseStd
    problem['dof'] = MeasurementDimension
    problem['type'] = 'linear-estimation'
    num_outliers = np.floor(outliers_percentage*N)
    problem['x_gt'] = np.random.normal(StateMeanandStd[0], StateMeanandStd[1], (StateDimension,1))
    
    problem['A'] = []
    for i in range(N):
        problem['A'].append(random_matrix(MeasurementDimension, StateDimension, min(MeasurementDimension, StateDimension)))
    #print('MeasurementDimension: ', MeasurementDimension)
    problem['y'] = []
    for i in range(N):
        problem['y'].append(problem['A'][i] .dot( problem['x_gt']) +  np.random.normal(0, int(MeasurementNoiseStd), size=(int(MeasurementDimension), 1)))
    
    convi = 1/MeasurementNoiseStd*np.eye(MeasurementDimension)
    problem['outliers'] = sorted(random.sample(range(problem['N']), int(np.array(num_outliers))))
    
    for i in problem['outliers']:
        y_gt = problem['y'][i]
        d = 0
        #print(OutlierSpace, np.shape(OutlierSpace))
        #print(d, chi2.ppf(0.999, int(MeasurementDimension)), d < chi2.ppf(0.999, int(MeasurementDimension)))
        while (d < chi2.ppf(0.999, int(MeasurementDimension))):
            #y_meas = rand_in(MeasurementDimension, 1, range(OutlierSpace[0], OutlierSpace[1]+1))
            
            y_meas = rand_in(MeasurementDimension, 1, (OutlierSpace))

            d = (y_gt - y_meas).T @ convi @ (y_gt - y_meas)
            #print(d < chi2.ppf(0.999, int(MeasurementDimension)))
        
        problem['y'][i] = y_meas
        
    problem['priors'] = []
    
    return problem
        
    
    

In [12]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);


3 <class 'int'>
True
1 True
True


In [13]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    print('Weights: ', Weights)
    print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [14]:
problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

3 <class 'int'>
True
1 True
True


In [15]:
def printProblemSummary(problem):
    num_outliers = np.size(problem['outliers']);
    print('Problem:\n');
    print('Type:', problem['type']);
    print('Measurements (N):', problem['N']);
    print('Num. outliers (k): ', num_outliers);
    print('Num. priors:   ', len(problem['priors']));
    print('Num. possible outliers (|V|): ', problem['N'] - np.size(problem['priors']));
    print('   k/N: %.03f\n', num_outliers/problem['N']);
    print('   k/|V|: %.03f\n', num_outliers/(problem['N'] - len(problem['priors'])));
    

In [16]:
printProblemSummary(problem)

Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6


In [17]:
print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
100
[-0.65854482 -0.49432533  0.90470577] (3,)


In [18]:
print(f_val_ls)

597313.6719750067


In [19]:
import numpy as np
from scipy.stats import chi2

cfg                 = {}
cfg['epsilon']      = chi2.ppf(0.99, problem['dof']) * np.array(problem['MeasurementNoiseStd'])**2
cfg['epsilon_mts']  = lambda n: chi2.ppf(0.99, n*problem['dof'])
cfg['epsilon_dmts'] = lambda n1, n2: UDGinv(0.99, n1*problem['dof']/2, n2*problem['dof']/2, 2*problem['MeasurementNoiseStd']**2, 2*problem['MeasurementNoiseStd']**2)
results             = {}
print('Solving with:')

Solving with:


In [20]:
print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime


Solving with:



In [21]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    print('Weights: ', Weights)
    print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    """"
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    """
    
    if 'Estimate' in kwargs and kwargs['Estimate'] is not None:
        x = kwargs['Estimate']
    else:
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [22]:
print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

  - Ground-Truth Least-Square...



In [23]:
start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

Inliers:  [2, 5, 6, 7, 8, 11, 12, 18, 22, 24, 25, 30, 32, 34, 35, 39, 44, 46, 47, 51, 52, 53, 54, 59, 64, 68, 70, 71, 73, 75, 76, 77, 79, 80, 82, 85, 86, 87, 89, 93]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
40
[0.9967754  0.83202013 1.39601747] (3,)


In [24]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))

[ 2  5  6  7  8 11 12 18 22 24 25 30 32 34 35 39 44 46 47 51 52 53 54 59
 64 68 70 71 73 75 76 77 79 80 82 85 86 87 89 93]
(array([ 2,  5,  6,  7,  8, 11, 12, 18, 22, 24, 25, 30, 32, 34, 35, 39, 44,
       46, 47, 51, 52, 53, 54, 59, 64, 68, 70, 71, 73, 75, 76, 77, 79, 80,
       82, 85, 86, 87, 89, 93], dtype=int64),)


In [25]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))
print(info_gt['residuals']<8)

[ 2  5  6  7  8 11 12 18 22 24 25 30 32 34 35 39 44 46 47 51 52 53 54 59
 64 68 70 71 73 75 76 77 79 80 82 85 86 87 89 93]
(array([ 2,  5,  6,  7,  8, 11, 12, 18, 22, 24, 25, 30, 32, 34, 35, 39, 44,
       46, 47, 51, 52, 53, 54, 59, 64, 68, 70, 71, 73, 75, 76, 77, 79, 80,
       82, 85, 86, 87, 89, 93], dtype=int64),)
[False False  True False False  True  True  True  True False False  True
  True False False False False False  True False False False  True False
  True  True False False False False  True False  True False  True  True
 False False False  True False False False False  True False  True  True
 False False False  True  True  True  True False False False False  True
 False False False False  True False False False  True False  True  True
 False  True False  True  True  True False  True  True False  True False
 False  True  True  True False  True False False False  True False False
 False False False False]


In [26]:
print(f_val_ls)
print(f_val_gt)

597313.6719750067
6.581318620839172e-28


In [27]:
def detectionStats(problem, inliers):
    num_outliers = len(problem['outliers'])
    if num_outliers == 0:
        true_positive_rate = 1
        false_positive_rate = 0
        precision = 1
        recall = 1
    else:
        outliers = np.setdiff1d(np.arange(1, problem['N']+1), inliers)
        true_inliers = np.setdiff1d(np.arange(1, problem['N']+1), np.union1d(problem['priors'], problem['outliers']))
        if len(true_inliers) == 0:
            true_positive_rate = 0
            false_positive_rate = 0
            precision = 0
            recall = 0
        else:
            true_positive = np.intersect1d(problem['outliers'], outliers)
            false_positive = np.intersect1d(true_inliers, outliers)
            true_positive_rate = len(true_positive) / num_outliers
            false_positive_rate = len(false_positive) / len(true_inliers)
            precision = len(true_positive) / (len(true_positive) + len(false_positive))
            recall = len(true_positive) / num_outliers
    return true_positive_rate, false_positive_rate, precision, recall


In [28]:
#tp, fp = detectionStats(problem, range(problem['N']))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}

In [29]:
detectionStats(problem, range(problem['N']))

(0.0, 0.024390243902439025, 0.0, 0.0)

In [30]:
results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

In [31]:
print(results['gt']['f_val'])
print(results['ls']['f_val'])

6.581318620839172e-28
597313.6719750067


In [32]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    # GNC - GNC weights update for the TLS cost function
    # Reference:
    #    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    #      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    #      https://arxiv.org/pdf/1909.08605.pdf
    #    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications"
    #      Antonante, Tzoumas, Yang, Carlone (2020).
    #      https://arxiv.org/pdf/2007.15109.pdf

    # Author: Pasquale Antonante
    # email: antonap@mit.edu
    # Date: 2021-01-06

    th1 = (mu+1)/mu * barc2
    th2 = (mu)/(mu+1) * barc2  # th1 > th2
    
    print('th1 and th2 are: \n', th1, th2)
    
    display(np.shape(np.array(residuals)))
    display(np.size(np.array(residuals)))
    
    for k in range(np.size(residuals)):
    #for k in range(np.shape(residuals)[1]):
        print('Residual[k]', residuals[k])
        #residuals_k = np.squeeze(residuals[k])
        #display(k, np.shape(residuals), np.shape(residuals[k]), np.shape(th1), np.shape(th2))
        #display(residuals[k])
        #display('Residuals[k] is: ', residuals[k])
        if (residuals[k] - th1) >= 0:
            weights[k] = 0
            print('residuals(k) - th1 >= 0: \n', residuals[k], th1 )
        elif (residuals[k] - th2) <= 0:
            weights[k] = 1
            print('residuals(k) - th2 <= 0: \n', residuals[k], th2 )
        else:
            weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu
            
            print('weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu', residuals[k], weights[k])
            assert 0 <= weights[k] <= 1, f'weights {weights[k]} calculation wrong!'
    
    display(type(weights))

    return weights, th1, th2


In [33]:
import math  
def gnc_vanilla(problem, f, **kwargs):
    continuation_factor = kwargs.get('ContinuationFactor', 1.4)
    initial_mu = kwargs.get('InitialMu', 'auto')
    noise_bound = kwargs.get('NoiseBound', 0)
    max_iterations = kwargs.get('MaxIterations', 1e3)
    fix_priors_weights = kwargs.get('FixPriorsWeights', True)
    cost_threshold = kwargs.get('CostThreshold', 0)
    debug = kwargs.get('Debug', False)
    init_ = kwargs.get('init_', 0)

    max_iterations = int(max_iterations)

    if debug:
        residuals_history = []
        weights_history = []

    if 'init_' not in kwargs:
        try:
            _, f_info = f(problem)
            #print('f(problem): ', f(problem))
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        assert 'residuals' in f_info, 'f should compute residuals'
    else:
        f_info = kwargs['init_']

    barc2 = noise_bound
    weights = np.ones(problem['N'])
    
    print('barc2 is: {}', barc2)
    print('Max of residuals: {}', np.max(f_info['residuals']))
    print('Residuals are: {}', f_info['residuals'])
    display(type(weights))
    
    
    if isinstance(initial_mu, str) and initial_mu.lower() == 'auto':
        mu = 1 / (2 * np.max(f_info['residuals']) / barc2 - 1 )
        display('mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )')
    elif isinstance(initial_mu, (int, float)):
        mu = initial_mu
    else:
        raise ValueError('Invalid value for InitialMu')

    prev_f_cost = 0

    i = 1
    info = {}
    info['stopping'] = 'MaxIterations'  # Worst case stopping
    info['status'] = 1  # worst case status: failure

    while i < max_iterations:
        if debug:
            residuals_history.append(f_info['residuals'].tolist())
            weights_history.append(weights.tolist())
            
        print('mu is: {}',mu)
        #print('f_info[residuals]')
        #print(f_info['residuals'])
        
        weights, th1out, th2out = gncWeightsUpdate(weights, mu, f_info['residuals'], barc2)
        print('Weights after gnc algorithm')
        print(weights)
        
        #print(type(weights))
        #print(type(list(weights)))
        
        if fix_priors_weights:
            weights[problem['priors']] = 1
        
        try:
            _, f_info = f(problem, Weights=weights)
            print('_, f_info = f(problem, Weights=weights)')
            #print(weights)
            
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        
        f_cost = np.sum(f_info['residuals'] * weights)
        
        cost_diff = np.abs(f_cost - prev_f_cost)
        
        prev_f_cost = f_cost

        if (cost_diff < cost_threshold) or areBinaryWeights(weights):
            if debug:
                residuals_history.append(f_info['residuals'].tolist())
                weights_history.append(weights.tolist())
            info['residuals'] = f_info['residuals'].tolist()
            info['stopping'] = 'CostThreshold'
            info['status'] = 1
            break
        mu *= continuation_factor
        i += 1
    
    
    #print(np.where(weights > 1-2.2204e-16))
    
    inliers = np.where(weights > 1-2.2204e-16)[0].tolist()
    
    info['Iterations'] = i
    info['params'] = kwargs
    info['Algorithm'] = 'GNC'
    if debug:
        info['mu'] = mu
        info['barc2History'] = np.repeat(barc2, i+1)
        info['ResidualsHistory'] = residuals_history
        info['WeightsHistory'] = weights_history
        info['CostDiff'] = cost_diff
        info['AreBinaryWeights'] = areBinaryWeights(weights)

    return inliers, info


In [34]:
def gnc(problem, f, **kwargs):
    """
    %GNC - Graduated Non-Convexity
    % Implementation of: 
    %    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    %      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    %      https://arxiv.org/pdf/1909.08605.pdf
    %    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications" 
    %      Antonante, Tzoumas, Yang, Carlone (2020).
    %      https://arxiv.org/pdf/2007.15109.pdf
    %
    % Syntax:  [inliers, info] = gnc(problem, f, ...)
    %
    % Inputs:
    %    problem - the stracture representing a generic problem (see EmptyProblem)
    %    f - a function_handler of a non-minimal solver for problem
    %
    % Options:
    %    - ContinuationFactor: continuation factor for mu
    %    - InitialMu: initial mu (default: auto)
    %    - NoiseBound: inlier threshold
    %    - MaxIterations: maximum number of iterations
    %    - FixPriorsWeights: fix priors weights to 1 across all iterations
    %    - CostThreshold: if weighted sum of squared residuals is below this value, GNC stops
    %    - Debug: whether or not to enable the debug information
    %
    % Outputs:
    %    inliers - the indices of the inliers
    %    info - structure containing extended information about the xecution
    % 
    % Example:
    %   problem = linearRegressionProblem(100, 0.8);
    %   [inliers, info] = gnc(problem, @leastSquareNorm2, ...
    %       'NoiseBound', chi2inv(0.99, problem.dof)*problem.MeasurementNoiseStd^2);

    % Author: Pasquale Antonante
    % email: antonap@mit.edu
    % Date: 2021-01-06
    """
    
    #inliers = kwargs.get('Inliers', measurements)
    #assert all(isinstance(elem, numbers.Number) for elem in inliers) and all(np.floor(inliers)==inliers), "Inliers must be numeric and integers"
    #print(inliers)
    #weights = kwargs.get('Weights', default_weights)
    
    
    #params = inputParser;
    #params.KeepUnmatched = true;
    #params.addParameter('NoiseBound', 0, @(x) isscalar(x) && x>=0 && isfinite(x));
    #arams.parse(varargin{:});
    #start_t = now;
    #if params.Results.NoiseBound > 0
    #    [inliers, info] = gnc_vanilla(problem, f, varargin{:});
    #else
    #    error('You need to set the Noise Bound')
    #end
    #end_t = now;
    
    NoiseBound = kwargs.get('NoiseBound', 0)
    assert(np.size(NoiseBound)==1 and NoiseBound >= 0 and np.isfinite(NoiseBound))
    
    assert isProblem(problem), 'The problem doesn''t contain required fields.'

    start_t = datetime.datetime.now();
    
    if NoiseBound > 0:
        [inliers, info] = gnc_vanilla(problem, f, **kwargs);
    else:
        ValueError('You need to set the Noise Bound')
    
    print('Output of gnc_vanilla')
    print(inliers)
    end_t = datetime.datetime.now();
    
    info = {}
    info['time'] = (end_t - start_t) * 24 * 60 * 60; #% serial date number to sec
    
    return inliers, info
    


In [35]:
def areBinaryWeights(weights):
    # Check the input vector is (approximately) a binary vector
    flag = True
    th = 1e-12
    for i in range(len(weights)-1, -1, -1):
        if weights[i] > th and abs(1 - weights[i]) > th:
            flag = False
            break
    return flag

In [36]:
print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
100
[-0.65854482 -0.49432533  0.90470577] (3,)
barc2 is: {} 0.11344866730144375
Max of residuals: {} 21079.394269175376
Residuals are: {} [11791.11086423  6986.24468239    87.49793591 10711.86954281
  9813.11753729    46.39172349    56.3973811     22.41523255
    73.55700362  3314.33433534 10850.02259996   107.62785435
    76.37994842  9213.96043891  3380.35357374  7317.02677803
  9060.28067464 19257.19804902    88.54507337  1976.42041796
 10410.59668502  8601.55910821    44.42675872 10186.78220306
    58.23671721   115.08748889 13534.9145022  10593.55051561
 13138.95544542 16788.74725712    64.1322534

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.690992234934003e-06
th1 and th2 are: 
 42158.78853835075 3.0528866124237945e-07


(100,)

...

100

Residual[k] 13916.34023338014
residuals(k) - th1 >= 0: 
 13916.34023338014 271.11165648438237
Residual[k] 7413.964193571432
residuals(k) - th1 >= 0: 
 7413.964193571432 271.11165648438237
Residual[k] 2.35699657157979e-10
residuals(k) - th2 <= 0: 
 2.35699657157979e-10 4.747342950639636e-05
Residual[k] 11848.42885452306
residuals(k) - th1 >= 0: 
 11848.42885452306 271.11165648438237
Residual[k] 9772.218252067485
residuals(k) - th1 >= 0: 
 9772.218252067485 271.11165648438237
Residual[k] 7.937939153008954e-11
residuals(k) - th2 <= 0: 
 7.937939153008954e-11 4.747342950639636e-05
Residual[k] 1.507263514581209e-10
residuals(k) - th2 <= 0: 
 1.507263514581209e-10 4.747342950639636e-05
Residual[k] 5.7523637469054076e-11
residuals(k) - th2 <= 0: 
 5.7523637469054076e-11 4.747342950639636e-05
Residual[k] 1.8384739718395678e-10
residuals(k) - th2 <= 0: 
 1.8384739718395678e-10 4.747342950639636e-05
Residual[k] 4438.180905341067
residuals(k) - th1 >= 0: 
 4438.180905341067 271.11165648438237
Res

 5304.701303420939 271.11165648438237
Residual[k] 11669.095188260031
residuals(k) - th1 >= 0: 
 11669.095188260031 271.11165648438237
Residual[k] 2975.998703273749
residuals(k) - th1 >= 0: 
 2975.998703273749 271.11165648438237


numpy.ndarray

Weights after gnc algorithm
[0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0.]
Inliers:  range(0, 100)
Weights:  [0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0.]
Type of Weights:  <class 'numpy.ndarray'>
100
[0.9967754  0.83202013 1.39601747] (3,)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[2, 5, 6, 7, 8, 11, 12, 18, 22, 24, 25, 30, 32, 34, 35, 39, 44, 46, 47, 51, 52, 53, 54, 59, 64, 68, 70, 71, 73, 75, 76, 77, 79, 80, 82, 85, 86, 87, 89, 93]
Inliers:  [2, 5, 6, 7,

In [37]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

597313.6719750067
6.581318620839172e-28
6.581318620839172e-28


In [38]:
print(inliers_gnc)

[2, 5, 6, 7, 8, 11, 12, 18, 22, 24, 25, 30, 32, 34, 35, 39, 44, 46, 47, 51, 52, 53, 54, 59, 64, 68, 70, 71, 73, 75, 76, 77, 79, 80, 82, 85, 86, 87, 89, 93]


In [39]:
print(inliers_gt)

[ 2  5  6  7  8 11 12 18 22 24 25 30 32 34 35 39 44 46 47 51 52 53 54 59
 64 68 70 71 73 75 76 77 79 80 82 85 86 87 89 93]


In [40]:
print(inliers_gnc == inliers_gt)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [41]:
print(info_gt['residues'])

[[ 8.75177126e+00  5.78365910e+00 -1.77635684e-15  1.06733361e+02
   7.74905088e+01  2.22044605e-15 -1.77635684e-15 -8.88178420e-16
  -4.44089210e-15  2.70463210e+01  6.22632204e+01  0.00000000e+00
  -8.88178420e-16 -2.92189247e+01  4.55217513e+01  6.66058424e+01
   8.02293767e+01 -7.83107494e+01 -1.77635684e-15 -2.73097786e+01
  -5.87485908e+01 -8.42070424e+00 -1.77635684e-15  3.07340551e+01
  -3.55271368e-15  1.77635684e-15 -6.09362233e+01  1.90289445e-01
  -6.93806335e+01  9.28211479e+01 -1.77635684e-15  2.12606198e+01
  -1.77635684e-15 -8.65085129e+01  2.22044605e-16 -4.44089210e-15
   4.84148332e+01 -5.09837547e+01 -6.95771704e+01  0.00000000e+00
   2.55565818e+00 -1.26151843e+01  1.66667205e+01  2.36487627e+01
  -3.55271368e-15  9.02707490e+01  1.77635684e-15  8.88178420e-16
  -5.73203627e+01  8.73295014e+00 -2.15595044e+01 -1.77635684e-15
  -1.11022302e-16  2.66453526e-15 -1.33226763e-15 -3.58304208e+01
   2.87191336e+01 -5.71712906e+01 -8.78819571e+01 -8.88178420e-16
   9.37410

In [42]:
print(info_gt['residuals'])

[1.39163367e+04 7.41396360e+03 4.25984889e-29 1.18484271e+04
 9.77221814e+03 2.07075988e-29 3.54987407e-30 1.38543696e-29
 3.94430453e-29 4.43817898e+03 1.09579641e+04 1.02551918e-29
 3.35265885e-30 7.77190804e+03 4.61433959e+03 9.33798800e+03
 1.01108247e+04 1.64640867e+04 9.46633086e-30 1.29736008e+03
 1.00212426e+04 7.20860879e+03 1.91298770e-29 1.23990485e+04
 1.99187379e-29 6.31088724e-30 1.44194227e+04 1.16015500e+04
 1.37345306e+04 1.69372711e+04 3.35265885e-30 9.69555472e+03
 7.09974815e-30 1.58763822e+04 2.94343725e-29 3.94430453e-29
 5.59620339e+03 3.40141657e+03 1.13104403e+04 7.09974815e-30
 1.44691975e+04 3.06354686e+02 8.54277694e+03 6.57070922e+03
 1.34106354e-29 1.25049360e+04 8.08582428e-30 2.02145607e-30
 7.58678989e+03 5.04432641e+03 1.34584186e+03 4.73316543e-30
 1.02675177e-29 3.17516514e-29 3.35265885e-30 1.28714274e+04
 9.72636209e+03 1.02842262e+04 1.30115377e+04 2.05103835e-29
 2.22710555e+04 1.37707470e+04 1.08439498e+04 6.67357712e+03
 1.41994963e-29 5.939624

In [42]:
print(info_gt['residuals'])

[1.39163367e+04 7.41396360e+03 4.25984889e-29 1.18484271e+04
 9.77221814e+03 2.07075988e-29 3.54987407e-30 1.38543696e-29
 3.94430453e-29 4.43817898e+03 1.09579641e+04 1.02551918e-29
 3.35265885e-30 7.77190804e+03 4.61433959e+03 9.33798800e+03
 1.01108247e+04 1.64640867e+04 9.46633086e-30 1.29736008e+03
 1.00212426e+04 7.20860879e+03 1.91298770e-29 1.23990485e+04
 1.99187379e-29 6.31088724e-30 1.44194227e+04 1.16015500e+04
 1.37345306e+04 1.69372711e+04 3.35265885e-30 9.69555472e+03
 7.09974815e-30 1.58763822e+04 2.94343725e-29 3.94430453e-29
 5.59620339e+03 3.40141657e+03 1.13104403e+04 7.09974815e-30
 1.44691975e+04 3.06354686e+02 8.54277694e+03 6.57070922e+03
 1.34106354e-29 1.25049360e+04 8.08582428e-30 2.02145607e-30
 7.58678989e+03 5.04432641e+03 1.34584186e+03 4.73316543e-30
 1.02675177e-29 3.17516514e-29 3.35265885e-30 1.28714274e+04
 9.72636209e+03 1.02842262e+04 1.30115377e+04 2.05103835e-29
 2.22710555e+04 1.37707470e+04 1.08439498e+04 6.67357712e+03
 1.41994963e-29 5.939624

In [46]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 0.06446302]
 [-1.56244413]
 [ 0.29887328]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [0, 1, 2, 6, 8, 19, 24, 25, 26, 27, 29, 36, 39, 40, 42, 43, 45, 47, 55, 56, 58, 63, 66, 67, 68, 69, 70, 76, 78, 80, 81, 82, 83, 85, 88, 92, 93, 94, 95, 96]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1

In [47]:
print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

638582.5174403916
9.174918402766909e-29
9.174918402766909e-29
638582.5174403916


In [48]:
print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 0.06446302]
 [-1.56244413]
 [ 0.29887328]] (3, 1)
barc2 is: {} 0.11344866730144375
Max of residuals: {} 27568.621705252655
Residuals are: {} [1.15555666e+01 2.45290648e-01 1.39930475e+01 1.10287727e+04
 4.43736366e+03 1.89166930e+03 1.79364304e+00 9.65631817e+03
 8.32035711e+00 1.22030135e+04 7.12074656e+03 1.95879620e+04
 2.70677139e+03 6.27208743e+03 9.56341566e+03 9.99180787e+03
 6.48660890e+03 2.75686217e+04 7.50274632e+03 5.74514931e+00
 6.36214623e+03 8.61188829e+03 3.14585403e+03 1.30121916e+04
 8.29142052e+00 3.04966833e+00 1.83226059e+01 1.06140341e+01
 1.44222947e+04 6.28129211e+00 8.84515

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.0575729527446357e-06
th1 and th2 are: 
 55137.2434105053 2.3342842906835337e-07


(100,)

...

(100,)

100

Residual[k] 4.263209229377505e-11
residuals(k) - th2 <= 0: 
 4.263209229377505e-11 7.11309918379761e-05
Residual[k] 3.093532485121872e-11
residuals(k) - th2 <= 0: 
 3.093532485121872e-11 7.11309918379761e-05
Residual[k] 5.462944715407854e-11
residuals(k) - th2 <= 0: 
 5.462944715407854e-11 7.11309918379761e-05
Residual[k] 10800.500355738428
residuals(k) - th1 >= 0: 
 10800.500355738428 180.94222757065776
Residual[k] 4479.701168028206
residuals(k) - th1 >= 0: 
 4479.701168028206 180.94222757065776
Residual[k] 1921.0866191525752
residuals(k) - th1 >= 0: 
 1921.0866191525752 180.94222757065776
Residual[k] 2.877351083037717e-11
residuals(k) - th2 <= 0: 
 2.877351083037717e-11 7.11309918379761e-05
Residual[k] 9766.402373360277
residuals(k) - th1 >= 0: 
 9766.402373360277 180.94222757065776
Residual[k] 8.144305865784656e-11
residuals(k) - th2 <= 0: 
 8.144305865784656e-11 7.11309918379761e-05
Residual[k] 12623.051399413735
residuals(k) - th1 >= 0: 
 12623.051399413735 180.94222757065776
Resi

numpy.ndarray

Weights after gnc algorithm
[1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 0. 0.]
Inliers:  range(0, 100)
Weights:  [1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 0. 0.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 0.36894649]
 [-0.50822799]
 [-0.63228875]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[0, 1, 2, 6, 8, 19, 24, 25, 26, 27, 29, 36, 39, 40, 42, 43, 45, 47, 55, 56, 58, 63, 66, 67, 68, 69, 70, 76, 78, 80, 81, 82, 83, 85, 88, 92, 93, 94, 95, 96]
Inliers:  [0,

In [49]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

638582.5174403916
9.174918402766909e-29
9.174918402766909e-29


In [50]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.7;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  70
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.7
   k/|V|: %.03f
 0.7
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 3.39141362]
 [-4.36283667]
 [ 5.66007129]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [0, 2, 4, 14, 15, 19, 25, 33, 36, 37, 38, 40, 41, 44, 48, 51, 57, 61, 66, 69, 78, 80, 82, 83, 87, 92, 94, 95, 96, 98]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 1.9613253639379623e-06
th1 and th2 are: 
 57842.97286780047 2.225093122701231e-07


(100,)

...

(100,)

100

Residual[k] 6.09795210661869e-11
residuals(k) - th2 <= 0: 
 6.09795210661869e-11 1.7658178565516807e-05
Residual[k] 4199.263455760682
residuals(k) - th1 >= 0: 
 4199.263455760682 728.8747287677563
Residual[k] 9.730060120116982e-12
residuals(k) - th2 <= 0: 
 9.730060120116982e-12 1.7658178565516807e-05
Residual[k] 6418.489411088903
residuals(k) - th1 >= 0: 
 6418.489411088903 728.8747287677563
Residual[k] 3.7674636814452977e-11
residuals(k) - th2 <= 0: 
 3.7674636814452977e-11 1.7658178565516807e-05
Residual[k] 7441.777827512365
residuals(k) - th1 >= 0: 
 7441.777827512365 728.8747287677563
Residual[k] 20658.96368968382
residuals(k) - th1 >= 0: 
 20658.96368968382 728.8747287677563
Residual[k] 16137.884219660104
residuals(k) - th1 >= 0: 
 16137.884219660104 728.8747287677563
Residual[k] 11274.149157530224
residuals(k) - th1 >= 0: 
 11274.149157530224 728.8747287677563
Residual[k] 22658.84312081058
residuals(k) - th1 >= 0: 
 22658.84312081058 728.8747287677563
Residual[k] 7851.9844463404

numpy.ndarray

Weights after gnc algorithm
[1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0.]
Inliers:  range(0, 100)
Weights:  [1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0.]
Type of Weights:  <class 'numpy.ndarray'>
[[3.20771095]
 [0.68367645]
 [2.31546764]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[0, 2, 4, 14, 15, 19, 25, 33, 36, 37, 38, 40, 41, 44, 48, 51, 57, 61, 66, 69, 78, 80, 82, 83, 87, 92, 94, 95, 96, 98]
Inliers:  [0, 2, 4, 14, 15, 19, 25, 33, 36, 37, 38, 40

In [55]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 2, 4, 14, 15, 19, 25, 33, 36, 37, 38, 40, 41, 44, 48, 51, 57, 61, 66, 69, 78, 80, 82, 83, 87, 92, 94, 95, 96, 98]
[ 0  2  4 14 15 19 25 33 36 37 38 40 41 44 48 51 57 61 66 69 78 80 82 83
 87 92 94 95 96 98]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
753658.159854078
1.1793963571119888e-27
1.1793963571119888e-27


In [56]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.8;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  80
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.8
   k/|V|: %.03f
 0.8
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 3.46941076]
 [ 0.11973153]
 [-2.90425683]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [1, 2, 4, 5, 12, 18, 20, 31, 38, 51, 56, 57, 61, 65, 66, 68, 79, 87, 98, 99]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.0670440682060667e-06
th1 and th2 are: 
 54884.60722721691 2.3450291006348367e-07


(100,)

...

(100,)

100

Residual[k] 15040.510564037533
residuals(k) - th1 >= 0: 
 15040.510564037533 352.9138964824738
Residual[k] 8.029405014552494e-10
residuals(k) - th2 <= 0: 
 8.029405014552494e-10 3.646951916814884e-05
Residual[k] 1.2316559526673323e-10
residuals(k) - th2 <= 0: 
 1.2316559526673323e-10 3.646951916814884e-05
Residual[k] 8152.291805457632
residuals(k) - th1 >= 0: 
 8152.291805457632 352.9138964824738
Residual[k] 7.260947202158353e-11
residuals(k) - th2 <= 0: 
 7.260947202158353e-11 3.646951916814884e-05
Residual[k] 4.3042082221209756e-10
residuals(k) - th2 <= 0: 
 4.3042082221209756e-10 3.646951916814884e-05
Residual[k] 819.652730368211
residuals(k) - th1 >= 0: 
 819.652730368211 352.9138964824738
Residual[k] 11516.806787850743
residuals(k) - th1 >= 0: 
 11516.806787850743 352.9138964824738
Residual[k] 1767.8190361873826
residuals(k) - th1 >= 0: 
 1767.8190361873826 352.9138964824738
Residual[k] 2898.3968919133054
residuals(k) - th1 >= 0: 
 2898.3968919133054 352.9138964824738
Residual[k] 

numpy.ndarray

Weights after gnc algorithm
[0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1.]
Inliers:  range(0, 100)
Weights:  [0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[-0.26274886]
 [ 1.24342027]
 [ 1.46088082]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[1, 2, 4, 5, 12, 18, 20, 31, 38, 51, 56, 57, 61, 65, 66, 68, 79, 87, 98, 99]
Inliers:  [1, 2, 4, 5, 12, 18, 20, 31, 38, 51, 56, 57, 61, 65, 66, 68, 79, 87, 98, 99]
Weigh

In [57]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[1, 2, 4, 5, 12, 18, 20, 31, 38, 51, 56, 57, 61, 65, 66, 68, 79, 87, 98, 99]
[ 1  2  4  5 12 18 20 31 38 51 56 57 61 65 66 68 79 87 98 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
714635.6585588157
1.0615109555880242e-28
1.0615109555880242e-28


In [58]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.9;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  90
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.9
   k/|V|: %.03f
 0.9
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[ 2.70779628]
 [-5.41181318]
 [ 3.5372256 ]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [40, 45, 53, 60, 61, 70, 76, 85, 91, 99]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.2773547834190403e-06
th1 and th2 are: 
 49816.096503850684 2.5836227676889134e-07


(100,)

...

(100,)

100

Residual[k] 7493.626612394612
residuals(k) - th1 >= 0: 
 7493.626612394612 878.7965553369767
Residual[k] 15240.437559640257
residuals(k) - th1 >= 0: 
 15240.437559640257 878.7965553369767
Residual[k] 7152.243222638879
residuals(k) - th1 >= 0: 
 7152.243222638879 878.7965553369767
Residual[k] 934.9340505823898
residuals(k) - th1 >= 0: 
 934.9340505823898 878.7965553369767
Residual[k] 8567.356772664467
residuals(k) - th1 >= 0: 
 8567.356772664467 878.7965553369767
Residual[k] 10651.087268256697
residuals(k) - th1 >= 0: 
 10651.087268256697 878.7965553369767
Residual[k] 14447.376889081745
residuals(k) - th1 >= 0: 
 14447.376889081745 878.7965553369767
Residual[k] 4437.379101636975
residuals(k) - th1 >= 0: 
 4437.379101636975 878.7965553369767
Residual[k] 5421.030066703669
residuals(k) - th1 >= 0: 
 5421.030066703669 878.7965553369767
Residual[k] 12175.284800527634
residuals(k) - th1 >= 0: 
 12175.284800527634 878.7965553369767
Residual[k] 8004.270874862485
residuals(k) - th1 >= 0: 
 8004.

numpy.ndarray

Weights after gnc algorithm
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1.]
Inliers:  range(0, 100)
Weights:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[2.19058137]
 [1.29313453]
 [1.66209265]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[40, 45, 53, 60, 61, 70, 76, 85, 91, 99]
Inliers:  [40, 45, 53, 60, 61, 70, 76, 85, 91, 99]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [59]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[40, 45, 53, 60, 61, 70, 76, 85, 91, 99]
[40 45 53 60 61 70 76 85 91 99]
[ True  True  True  True  True  True  True  True  True  True]
958520.5209359778
1.9665131401753637e-28
1.9665131401753637e-28


In [60]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[-0.73026981]
 [ 4.61778617]
 [-3.63377243]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [0, 1, 2, 4, 5, 9, 10, 11, 14, 18, 19, 22, 25, 28, 29, 31, 33, 35, 36, 42, 43, 48, 50, 56, 59, 62, 63, 65, 66, 72, 73, 77, 80, 81, 84, 89, 90, 94, 96, 97]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.5808337175333433e-06
th1 and th2 are: 
 43958.26020206349 2.9279139013489664e-07


(100,)

...

(100,)

100

Residual[k] 1.604944536355042e-13
residuals(k) - th2 <= 0: 
 1.604944536355042e-13 6.0490696690790115e-06
Residual[k] 2.399261389682376e-13
residuals(k) - th2 <= 0: 
 2.399261389682376e-13 6.0490696690790115e-06
Residual[k] 2.118331462840602e-12
residuals(k) - th2 <= 0: 
 2.118331462840602e-12 6.0490696690790115e-06
Residual[k] 15000.113722526077
residuals(k) - th1 >= 0: 
 15000.113722526077 2127.699103593439
Residual[k] 1.863970076017456e-12
residuals(k) - th2 <= 0: 
 1.863970076017456e-12 6.0490696690790115e-06
Residual[k] 5.742837044129312e-13
residuals(k) - th2 <= 0: 
 5.742837044129312e-13 6.0490696690790115e-06
Residual[k] 12121.036590600772
residuals(k) - th1 >= 0: 
 12121.036590600772 2127.699103593439
Residual[k] 10637.127530222548
residuals(k) - th1 >= 0: 
 10637.127530222548 2127.699103593439
Residual[k] 11289.863532701207
residuals(k) - th1 >= 0: 
 11289.863532701207 2127.699103593439
Residual[k] 7.343815617473207e-13
residuals(k) - th2 <= 0: 
 7.343815617473207e-13 6.04906

numpy.ndarray

Weights after gnc algorithm
[1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0.]
Inliers:  range(0, 100)
Weights:  [1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0.]
Type of Weights:  <class 'numpy.ndarray'>
[[1.60025259]
 [0.87693801]
 [0.69144026]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[0, 1, 2, 4, 5, 9, 10, 11, 14, 18, 19, 22, 25, 28, 29, 31, 33, 35, 36, 42, 43, 48, 50, 56, 59, 62, 63, 65, 66, 72, 73, 77, 80, 81, 84, 89, 90, 94, 96, 97]
Inliers:  [0, 1, 

In [61]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 1, 2, 4, 5, 9, 10, 11, 14, 18, 19, 22, 25, 28, 29, 31, 33, 35, 36, 42, 43, 48, 50, 56, 59, 62, 63, 65, 66, 72, 73, 77, 80, 81, 84, 89, 90, 94, 96, 97]
[ 0  1  2  4  5  9 10 11 14 18 19 22 25 28 29 31 33 35 36 42 43 48 50 56
 59 62 63 65 66 72 73 77 80 81 84 89 90 94 96 97]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
626109.3068726604
3.751329427165368e-27
3.751329427165368e-27


In [62]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.5;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



3 <class 'int'>
True
1 True
True
3 <class 'int'>
True
1 True
True
Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  50
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.5
   k/|V|: %.03f
 0.5
  - Least-Square

Inliers:  range(0, 100)
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[-1.80597756]
 [ 0.45783841]
 [ 1.01892948]] (3, 1)
Solving with:

  - Ground-Truth Least-Square...

Inliers:  [0, 4, 7, 9, 10, 11, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 36, 40, 42, 45, 46, 48, 53, 54, 55, 56, 58, 60, 61, 62, 64, 66, 69, 71, 73, 77, 80, 82, 86, 89, 91, 92, 94, 99]
Weights:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

numpy.ndarray

'mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )'

mu is: {} 2.5359083369953606e-06
th1 and th2 are: 
 44737.009355505164 2.8769469166337707e-07


(100,)

...

(100,)

100

Residual[k] 4.387104301913771e-10
residuals(k) - th2 <= 0: 
 4.387104301913771e-10 4.47385791244136e-05
Residual[k] 333.567019922387
residuals(k) - th1 >= 0: 
 333.567019922387 287.6845971500748
Residual[k] 4451.822839071591
residuals(k) - th1 >= 0: 
 4451.822839071591 287.6845971500748
Residual[k] 18940.73166214232
residuals(k) - th1 >= 0: 
 18940.73166214232 287.6845971500748
Residual[k] 3.4445860122252687e-10
residuals(k) - th2 <= 0: 
 3.4445860122252687e-10 4.47385791244136e-05
Residual[k] 15447.212685470022
residuals(k) - th1 >= 0: 
 15447.212685470022 287.6845971500748
Residual[k] 19579.467445247894
residuals(k) - th1 >= 0: 
 19579.467445247894 287.6845971500748
Residual[k] 1.3067124921860521e-10
residuals(k) - th2 <= 0: 
 1.3067124921860521e-10 4.47385791244136e-05
Residual[k] 3057.819651929067
residuals(k) - th1 >= 0: 
 3057.819651929067 287.6845971500748
Residual[k] 4.296877208740166e-10
residuals(k) - th2 <= 0: 
 4.296877208740166e-10 4.47385791244136e-05
Residual[k] 1.187629

numpy.ndarray

Weights after gnc algorithm
[1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 1.]
Inliers:  range(0, 100)
Weights:  [1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 1.]
Type of Weights:  <class 'numpy.ndarray'>
[[1.34433284]
 [1.6471007 ]
 [1.97600477]] (3, 1)
_, f_info = f(problem, Weights=weights)
Output of gnc_vanilla
[0, 4, 7, 9, 10, 11, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 36, 40, 42, 45, 46, 48, 53, 54, 55, 56, 58, 60, 61, 62, 64, 66, 69, 71, 73, 77, 80, 82,

In [63]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 4, 7, 9, 10, 11, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 36, 40, 42, 45, 46, 48, 53, 54, 55, 56, 58, 60, 61, 62, 64, 66, 69, 71, 73, 77, 80, 82, 86, 89, 91, 92, 94, 99]
[ 0  4  7  9 10 11 13 14 15 18 19 20 21 22 23 24 25 26 27 29 30 32 36 40
 42 45 46 48 53 54 55 56 58 60 61 62 64 66 69 71 73 77 80 82 86 89 91 92
 94 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
496774.48388506024
6.234959379640572e-28
6.234959379640572e-28
